# Notebook for running EfficientNet experiments
Contributions: Christoph Nötzli, Erik Norén and Sushruth Badri <br>
Comments: Erik Norén 14/12-22# Testing EfficientNet

In [1]:
# imports
import tensorflow as tf
import data.imageReading as ir
from fairness import fairnessMetrics as fm
from model import model as m
from model import utils as utils
from model import biasMitigation as mit
from model import evaluation as ev
from tuning import callbacks as cb
import datetime
import os
import numpy as np

In [2]:
# image size selected according to EfficientNetB3 input size
image_size = (300,300)
batch_size = 128

# Load datasets from current _split folders
# Load the Museum data set
(ds_train, train_batches, ds_val, val_batches, ds_test, test_batches, count_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces_split", image_size, batch_size, None, False, False)
# Load the FairFace data set
(ffds_train, fftrain_batches, ffds_val, ffval_batches, ffds_test, fftest_batches, ffcount_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/FairFace_split", image_size, batch_size, None, False, False)


Found 3916 images belonging to 2 classes.
Found 489 images belonging to 2 classes.
Found 492 images belonging to 2 classes.
Count classes: (2052, 1864, 256, 233, 258, 234)


2022-12-16 13:30:08.121006: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38262 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:ca:00.0, compute capability: 8.0
2022-12-16 13:30:08.122630: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38262 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:e3:00.0, compute capability: 8.0


Found 78158 images belonging to 2 classes.
Found 9769 images belonging to 2 classes.
Found 9771 images belonging to 2 classes.
Count classes: (36736, 41422, 4592, 5177, 4592, 5179)


## Find learning rate

In [4]:
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(False, (300,300), "Efficient")
    metric_list = m.metrics_list()

model.compile(optimizer=tf.keras.optimizers.Adam(), 
                           loss="binary_crossentropy", 
                           metrics="accuracy")

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:local

In [5]:
epochs = 200
train_input = []
train_output = []
nr_batches = 10

model.fit(ds_train, steps_per_epoch=nr_batches, callbacks=[cb.LerningRateCallback(100, nr_batches)], epochs=epochs, verbose=2)

Epoch 1/200
INFO:tensorflow:batch_all_reduce: 342 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 342 all-reduces with algorithm = nccl, num_packs = 1


2022-12-12 18:23:42.144523: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-12 18:23:43.105495: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-12 18:23:47.323490: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


10/10 - 76s - loss: 0.9376 - accuracy: 0.4898 - 76s/epoch - 8s/step
Epoch 2/200
10/10 - 12s - loss: 0.9556 - accuracy: 0.4734 - 12s/epoch - 1s/step
Epoch 3/200
10/10 - 12s - loss: 0.9329 - accuracy: 0.4867 - 12s/epoch - 1s/step
Epoch 4/200
10/10 - 13s - loss: 0.8972 - accuracy: 0.5041 - 13s/epoch - 1s/step
Epoch 5/200
10/10 - 12s - loss: 0.9598 - accuracy: 0.4922 - 12s/epoch - 1s/step
Epoch 6/200
10/10 - 12s - loss: 0.9210 - accuracy: 0.5047 - 12s/epoch - 1s/step
Epoch 7/200
10/10 - 12s - loss: 0.9264 - accuracy: 0.5008 - 12s/epoch - 1s/step
Epoch 8/200
10/10 - 12s - loss: 0.9522 - accuracy: 0.4672 - 12s/epoch - 1s/step
Epoch 9/200
10/10 - 12s - loss: 0.9211 - accuracy: 0.5000 - 12s/epoch - 1s/step
Epoch 10/200
10/10 - 11s - loss: 0.9297 - accuracy: 0.5090 - 11s/epoch - 1s/step
Epoch 11/200
10/10 - 12s - loss: 0.9545 - accuracy: 0.4820 - 12s/epoch - 1s/step
Epoch 12/200
10/10 - 12s - loss: 0.9480 - accuracy: 0.4914 - 12s/epoch - 1s/step
Epoch 13/200
10/10 - 12s - loss: 0.9159 - accurac

# Museum baseline
Baseline experiments to find how the EfficientNet model performs on the Museum data set with and without bias mitigation methods

## Test EfficientNet Museum
Train and test EfficientNet model using Museum data set, with no pre- or in-processing bias mitigation methods applied

In [6]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # build model without augmentation
    model = m.build_model(False, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

# Train model without class weights
utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight=None, weight=False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_Museum"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/EfficientNet_Museum.h5")

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 342 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 342 all-reduces with algorithm = nccl, num_packs = 1
31/31 [==============================] - 183s 2s/step - loss: 0.7653 - accuracy: 0.5958 - true_positives_21: 1129.0000 - true_negatives_14: 1204.0000 - false_positives_17: 849.0000 - false_negatives_18: 734.0000 - true_positive_rate: 0.6060 - true_negative_rate: 0.5865 - false_positive_rate: 0.4135 - false_negative_rate: 0.3940 - positive_predicted_value: 0.5708 - false_discovery_rate: 0.4292 - negative_predicted_value: 0.6213 - false_omission_rate: 0.3787 - binary_demographic_parity_diff: 40.0000 - demographic_parity: 1978.0000 - binary_equalized_odds_diff: -0.0196 - binary_proportional_parity_diff: 0.0102 - proportional_parity: 0.5051 - binary_predictive_rate_parity_diff: -0.0505 - predictive_rate_parity: 0.5708 - binary_accuracy_parity_diff: 0.1915 - accuracy_parity: 0.5958 - false_negative_rate_parit

In [7]:
# Test the model
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
Instructions for updating:
use `experimental_local_results` instead.
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.789
    
    True positives: 184.000
    False positives: 54.000
    
    True negatives: 204.000
    False negatives: 50.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.786
    True negative rate tn/(tn+fp): 0.791
    
    False negative rate fn/(tp+fn): 0.214
    False positive rate fp/(tn+fp): 0.209
    
    Positive predicted value tp/(tp+fp): 0.773
    False discovery rate fp/(tp+fp): 0.227
    
    Negative predicted value tn/(tn+fn): 0.803
    False omission rate fn/(tn+fn): 0.197
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -16.000
    Demographic parity tp+fp: 238.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -0.004
    
    Binary proportion

In [8]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Build model without augmentation
    model = m.build_model(False, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Crossvalidate model using entire Museum data set
history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (300,300), None, 5, 20)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Museum_effnet_baseline"
os.mkdir(dir_name)

# Evaluate the results of the crossvalidation
ev.evaluateCrossValidation(dir_name, history)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Load data...
Start crossvalidation...
Fold 0...


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 342 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 342 all-reduces with algorithm = nccl, num_packs = 1
31/31 [==============================] - 186s 2s/step - loss: 0.7725 - accuracy: 0.5869 - true_positives_42: 1119.0000 - true_negatives_28: 1180.0000 - false_positives_34: 873.0000 - false_negatives_36: 745.0000 - true_positive_rate: 0.6003 - true_negative_rate: 0.5748 - false_positive_rate: 0.4252 - false_negative_rate: 0.3997 - positive_predicted_value: 0.5617 - false_discovery_rate: 0.4383 - negative_predicted_value: 0.6130 - false_omission_rate: 0.3870 - binary_demographic_parity_diff: 67.0000 - demographic_parity: 1992.0000 - binary_equalized_odds_diff: -0.0256 - binary_proportional_parity_diff: 0.0171 - proportional_parity: 0.5086 - binary_predictive_rate_parity_diff: -0.0512 - predictive_rate_parity: 0.5617 - binary_accuracy_parity_diff: 0.1739 - accuracy_parity: 0.5869 - false_negative_rate_parit

## Test EfficientNet Museum (weighted) 
Train and test EfficientNet model using Museum data set, with bias mitigation method reweighting applied

In [9]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Build model without augmentation
    model = m.build_model(False, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)


# Train model with class weights
utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_Museum_re"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/EfficientNet_Musuem_re.h5")

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Epoch 1/20
31/31 [==============================] - 193s 2s/step - loss: 0.7774 - accuracy: 0.5958 - true_positives_63: 1145.0000 - true_negatives_42: 1188.0000 - false_positives_51: 872.0000 - false_negatives_54: 711.0000 - true_positive_rate: 0.6169 - true_negative_rate: 0.5767 - false_positive_rate: 0.4233 - false_negative_rate: 0.3831 - positive_predicted_value: 0.5677 - false_discovery_rate: 0.4323 - negative_predicted_value: 0.6256 - false_omission_rate: 0.3744 - binary_demographic_parity_diff: 118.0000 - demographic_parity: 2017.0000 - binary_equalized_odds_diff: -0.0402 - binary_proportional_parity_diff: 0.0301 - proportional_parity: 0.5151 - binary_predictive_rate_parity_diff: -0.0579 - predictive_rate_parity: 0.5677 - binary_accuracy_parity_diff: 0.1915 - accuracy_parity: 0.5958 -

In [10]:
# Test the model
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.563
    
    True positives: 115.000
    False positives: 96.000
    
    True negatives: 162.000
    False negatives: 119.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.491
    True negative rate tn/(tn+fp): 0.628
    
    False negative rate fn/(tp+fn): 0.509
    False positive rate fp/(tn+fp): 0.372
    
    Positive predicted value tp/(tp+fp): 0.545
    False discovery rate fp/(tp+fp): 0.455
    
    Negative predicted value tn/(tn+fn): 0.577
    False omission rate fn/(tn+fn): 0.423
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -70.000
    Demographic parity tp+fp: 211.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -0.136
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.1

In [5]:
strategy = tf.distribute.MirroredStrategy()


with strategy.scope():
    # Build model without augmentation
    model = m.build_model(False, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Crossvalidate model using entire Museum data set
history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (300,300), class_weight, 5, 20)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Museum_re_effnet"
os.mkdir(dir_name)

# Evaluate the results of the crossvalidation
ev.evaluateCrossValidation(dir_name, history)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Load data...
Start crossvalidation...
Fold 0...


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 342 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 342 all-reduces with algorithm = nccl, num_packs = 1


2022-12-13 17:15:43.541279: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-13 17:15:45.151436: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-13 17:15:48.368501: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


31/31 [==============================] - 186s 2s/step - loss: 0.7729 - accuracy: 0.5882 - true_positives_21: 1120.0000 - true_negatives_14: 1184.0000 - false_positives_17: 869.0000 - false_negatives_18: 744.0000 - true_positive_rate: 0.6009 - true_negative_rate: 0.5767 - false_positive_rate: 0.4233 - false_negative_rate: 0.3991 - positive_predicted_value: 0.5631 - false_discovery_rate: 0.4369 - negative_predicted_value: 0.6141 - false_omission_rate: 0.3859 - binary_demographic_parity_diff: 61.0000 - demographic_parity: 1989.0000 - binary_equalized_odds_diff: -0.0241 - binary_proportional_parity_diff: 0.0156 - proportional_parity: 0.5078 - binary_predictive_rate_parity_diff: -0.0510 - predictive_rate_parity: 0.5631 - binary_accuracy_parity_diff: 0.1764 - accuracy_parity: 0.5882 - false_negative_rate_parity_diff: 0.0241 - false_positive_rate_parity_diff: -0.0241 - binary_negative_predictive_rate_parity_diff: 0.0510 - negative_predictive_rate_parity: 0.5631 - binary_specificity_parity_dif

## Test EfficientNet Museum augmented (unweighted) 
Train and test EfficientNet model using Museum data set, with bias mitigation method data set augmentation applied

In [6]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Build model with augmentation layers
    model = m.build_model(True, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

# Train model without class weights
utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, None, False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_Museum_aug"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/EfficientNet_Mueseum_aug.h5")

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Epoch 1/20
31/31 [==============================] - 206s 2s/step - loss: 0.7974 - accuracy: 0.5684 - true_positives_42: 1071.0000 - true_negatives_28: 1155.0000 - false_positives_34: 897.0000 - false_negatives_36: 793.0000 - true_positive_rate: 0.5746 - true_negative_rate: 0.5629 - false_positive_rate: 0.4371 - false_negative_rate: 0.4254 - positive_predicted_value: 0.5442 - false_discovery_rate: 0.4558 - negative_predicted_value: 0.5929 - false_omission_rate: 0.4071 - binary_demographic_parity_diff: 20.0000 - demographic_parity: 1968.0000 - binary_equalized_odds_diff: -0.0117 - binary_proportional_parity_diff: 0.0051 - proportional_parity: 0.5026 - binary_predictive_rate_parity_diff: -0.0487 - predictive_rate_parity: 0.5442 - binary_accuracy_parity_diff: 0.1369 - accuracy_parity: 0.5684 - 

In [7]:
# Test the model
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
Instructions for updating:
use `experimental_local_results` instead.
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.799
    
    True positives: 189.000
    False positives: 54.000
    
    True negatives: 204.000
    False negatives: 45.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.808
    True negative rate tn/(tn+fp): 0.791
    
    False negative rate fn/(tp+fn): 0.192
    False positive rate fp/(tn+fp): 0.209
    
    Positive predicted value tp/(tp+fp): 0.778
    False discovery rate fp/(tp+fp): 0.222
    
    Negative predicted value tn/(tn+fn): 0.819
    False omission rate fn/(tn+fn): 0.181
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -6.000
    Demographic parity tp+fp: 243.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.017
    
    Binary proportional

In [8]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Build model with augmentation layers
    model = m.build_model(True, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Crossvalidate model using entire Museum data set
history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (300,300), None, 5, 20)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Museum_aug_effnet"
os.mkdir(dir_name)

# Evaluate the results of the crossvalidation
ev.evaluateCrossValidation(dir_name, history)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Load data...
Start crossvalidation...
Fold 0...


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
31/31 [==============================] - 189s 2s/step - loss: 0.8234 - accuracy: 0.5525 - true_positives_63: 1024.0000 - true_negatives_42: 1140.0000 - false_positives_51: 911.0000 - false_negatives_54: 842.0000 - true_positive_rate: 0.5488 - true_negative_rate: 0.5558 - false_positive_rate: 0.4442 - false_negative_rate: 0.4512 - positive_predicted_value: 0.5292 - false_discovery_rate: 0.4708 - negative_predicted_value: 0.5752 - false_omission_rate: 0.4248 - binary_demographic_parity_diff: -47.0000 - demographic_parity: 1935.0000 - binary_equalized_odds_diff: 0.0071 - binary_proportional_parity_diff: -0.0120 - proportional_parity: 0.4940 - binary_predictive_rate_parity_diff: -0.0460 - predictive_rate_parity: 0.5292 - binary_accuracy_parity_diff: 0.1049 - accuracy_parity: 0.5525 - false_negative_rate_parity_diff: -0.0071 - false_positive_rate_parity_diff: 0.0071 - binary_negative_predictive_rate_parity_diff: 0.0460 - negative_predictive_rate_parity: 0.5292 - binary_specificit

# FairFace
For transfer learning EfficientNet model is trained on the FairFace data set in order to create a base model.

## Test EfficientNet FairFace (unweighted) 
Train and test EfficientNet model using Museum data set, with no pre- or in-processing bias mitigation methods applied

In [9]:
epochs = 20
class_weight = mit.findClassWeights(fftrain_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Build model without augmentation layers
    model = m.build_model(False, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

# Train model without class weights
utils.train_model(model, epochs, ffds_train, fftrain_batches, ffds_val, ffval_batches, None, False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_FairFace"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/EfficientNet_FairFace.h5")

Weight for class 0: 1.06
Weight for class 1: 0.94
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Epoch 1/20
611/611 [==============================] - 872s 1s/step - loss: 0.4156 - accuracy: 0.8034 - true_positives_84: 33901.0000 - true_negatives_56: 28891.0000 - false_positives_68: 7845.0000 - false_negatives_72: 7521.0000 - true_positive_rate: 0.8184 - true_negative_rate: 0.7864 - false_positive_rate: 0.2136 - false_negative_rate: 0.1816 - positive_predicted_value: 0.8121 - false_discovery_rate: 0.1879 - negative_predicted_value: 0.7934 - false_omission_rate: 0.2066 - binary_demographic_parity_diff: 5334.0000 - demographic_parity: 41746.0000 - binary_equalized_odds_diff: -0.0320 - binary_proportional_parity_diff: 0.0682 - proportional_parity: 0.5341 - binary_predictive_rate_parity_diff: 0.0186 - predictive_rate_parity: 0.8121 - binary_accuracy_parity_diff: 0.6068 - accuracy_parity: 0

In [10]:
# Test the model
test_predict, test_labels, dir_name = ev.testModel(model, ffds_test, fftest_batches, dir_name)
ev.testModelWithThresholdChange(model, ffds_val, ffval_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.926
    
    True positives: 4889.000
    False positives: 437.000
    
    True negatives: 4155.000
    False negatives: 290.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.944
    True negative rate tn/(tn+fp): 0.905
    
    False negative rate fn/(tp+fn): 0.056
    False positive rate fp/(tn+fp): 0.095
    
    Positive predicted value tp/(tp+fp): 0.918
    False discovery rate fp/(tp+fp): 0.082
    
    Negative predicted value tn/(tn+fn): 0.935
    False omission rate fn/(tn+fn): 0.065
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): 881.000
    Demographic parity tp+fp: 5326.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.039
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): 0

## Test EfficientNet FairFace (weighted) 
Train and test EfficientNet model using FairFace data set, with bias mitigation method reweighting applied

In [5]:
epochs = 20

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Build model without augmentation layers
    model = m.build_model(False, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

# Train model with class weights
utils.train_model(model, epochs, ffds_train, fftrain_batches, ffds_val, ffval_batches, class_weight)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_FairFace_re"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/EfficientNet_FairFace_re.h5")

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 342 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 342 all-reduces with algorithm = nccl, num_packs = 1


2022-12-13 20:51:23.632230: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-13 20:51:25.351663: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-13 20:51:28.812391: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


611/611 [==============================] - 869s 1s/step - loss: 0.4106 - accuracy: 0.8065 - true_positives_21: 34569.0000 - true_negatives_14: 28467.0000 - false_positives_17: 8269.0000 - false_negatives_18: 6853.0000 - true_positive_rate: 0.8346 - true_negative_rate: 0.7749 - false_positive_rate: 0.2251 - false_negative_rate: 0.1654 - positive_predicted_value: 0.8070 - false_discovery_rate: 0.1930 - negative_predicted_value: 0.8060 - false_omission_rate: 0.1940 - binary_demographic_parity_diff: 7518.0000 - demographic_parity: 42838.0000 - binary_equalized_odds_diff: -0.0596 - binary_proportional_parity_diff: 0.0962 - proportional_parity: 0.5481 - binary_predictive_rate_parity_diff: 9.9647e-04 - predictive_rate_parity: 0.8070 - binary_accuracy_parity_diff: 0.6130 - accuracy_parity: 0.8065 - false_negative_rate_parity_diff: 0.0596 - false_positive_rate_parity_diff: -0.0596 - binary_negative_predictive_rate_parity_diff: -9.9647e-04 - negative_predictive_rate_parity: 0.8070 - binary_speci

In [6]:
# Test the model
test_predict, test_labels, dir_name = ev.testModel(model, ffds_test, fftest_batches, dir_name)
ev.testModelWithThresholdChange(model, ffds_val, ffval_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
Instructions for updating:
use `experimental_local_results` instead.
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.925
    
    True positives: 4899.000
    False positives: 457.000
    
    True negatives: 4135.000
    False negatives: 280.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.946
    True negative rate tn/(tn+fp): 0.900
    
    False negative rate fn/(tp+fn): 0.054
    False positive rate fp/(tn+fp): 0.100
    
    Positive predicted value tp/(tp+fp): 0.915
    False discovery rate fp/(tp+fp): 0.085
    
    Negative predicted value tn/(tn+fn): 0.937
    False omission rate fn/(tn+fn): 0.063
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): 941.000
    Demographic parity tp+fp: 5356.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.045
    
    Binary propor

## Test EfficientNet FairFace augmented (unweighted) 
Train and test EfficientNet model using FairFace data set, with bias mitigation method data set augmentation applied

In [7]:
epochs = 20
class_weight = mit.findClassWeights(fftrain_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Build model with augmentation layers
    model = m.build_model(True, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

# Train model without class weights
utils.train_model(model, epochs, ffds_train, fftrain_batches, ffds_val, ffval_batches, None, False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_FairFace_aug"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/EfficientNet_FairFace_aug.h5")

Weight for class 0: 1.06
Weight for class 1: 0.94
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Epoch 1/20
INFO:tensorflow:batch_all_reduce: 342 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 342 all-reduces with algorithm = nccl, num_packs = 1
611/611 [==============================] - 854s 1s/step - loss: 0.4886 - accuracy: 0.7616 - true_positives_122: 32682.0000 - true_negatives_80: 26843.0000 - false_positives_98: 9893.0000 - false_negatives_104: 8740.0000 - true_positive_rate: 0.7890 - true_negative_rate: 0.7307 - false_positive_rate: 0.2693 - false_negative_rate: 0.2110 - positive_predicted_value: 0.7676 - false_discovery_rate: 0.2324 - negative_predicted_value: 0.7544 - false_omission_rate: 0.2456 - binary_demographic_parity_diff: 6992.0000 - demographic_parity: 42575.0000 - binary_equalized_odds_diff: -0.0583 - binary_proportional_parity_dif

In [13]:
# Test the model
test_predict, test_labels, dir_name = ev.testModel(model, ffds_test, fftest_batches, dir_name)
ev.testModelWithThresholdChange(model, ffds_val, ffval_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.915
    
    True positives: 4842.000
    False positives: 498.000
    
    True negatives: 4094.000
    False negatives: 337.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.935
    True negative rate tn/(tn+fp): 0.892
    
    False negative rate fn/(tp+fn): 0.065
    False positive rate fp/(tn+fp): 0.108
    
    Positive predicted value tp/(tp+fp): 0.907
    False discovery rate fp/(tp+fp): 0.093
    
    Negative predicted value tn/(tn+fn): 0.924
    False omission rate fn/(tn+fn): 0.076
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): 909.000
    Demographic parity tp+fp: 5340.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.043
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): 0

# Transfer Learning
Here the experiments for performing transfer learning are performed. The base EfficientNet models trained on the FairFace data set are loaded. These models then have some of their layers frozen and is then trained again using the Museum data set.

## Transfer learning without augmentation and reweighting
Transfer learning with EfficientNet, with no pre- or in-processing bias mitigation methods applied. Trained and tested with Museum data set.

In [4]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose the directory where model EfficientNet_FairFace.h5 exists
dir_name = "XXXXXXXX_XXXXXX_EfficientNet_Fairface"

with strategy.scope():
    # Load base model trained on FairFace data set without reweighting and without augmentation layers
    model = utils.loadModel(dir_name + "/EfficientNet_FairFace.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Freeze layers
utils.freezeCertainLayers(model, 334)    
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

# Train model without class weights
utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, None, False)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_transfer"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name_transfer)
utils.saveModel(model, dir_name_transfer + "/EfficientNet_transfer.h5")

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:local

/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 49 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 49 all-reduces with algorithm = nccl, num_packs = 1


2022-12-13 20:53:32.612344: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-13 20:53:34.325434: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-13 20:53:37.703724: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


31/31 [==============================] - 113s 1s/step - loss: 0.6679 - accuracy: 0.6987 - true_positives_40: 1499.0000 - true_negatives_26: 1237.0000 - false_positives_32: 815.0000 - false_negatives_34: 365.0000 - true_positive_rate: 0.8042 - true_negative_rate: 0.6028 - false_positive_rate: 0.3972 - false_negative_rate: 0.1958 - positive_predicted_value: 0.6478 - false_discovery_rate: 0.3522 - negative_predicted_value: 0.7722 - false_omission_rate: 0.2278 - binary_demographic_parity_diff: 712.0000 - demographic_parity: 2314.0000 - binary_equalized_odds_diff: -0.2014 - binary_proportional_parity_diff: 0.1818 - proportional_parity: 0.5909 - binary_predictive_rate_parity_diff: -0.1244 - predictive_rate_parity: 0.6478 - binary_accuracy_parity_diff: 0.3973 - accuracy_parity: 0.6987 - false_negative_rate_parity_diff: 0.2014 - false_positive_rate_parity_diff: -0.2014 - binary_negative_predictive_rate_parity_diff: 0.1244 - negative_predictive_rate_parity: 0.6478 - binary_specificity_parity_di

In [5]:
# Test the model
test_predict, test_labels, dir_name_transfer = ev.testModel(model, ds_test, test_batches, dir_name_transfer)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name_transfer)

Testing Model
-------------
Instructions for updating:
use `experimental_local_results` instead.
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.833
    
    True positives: 193.000
    False positives: 41.000
    
    True negatives: 217.000
    False negatives: 41.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.825
    True negative rate tn/(tn+fp): 0.841
    
    False negative rate fn/(tp+fn): 0.175
    False positive rate fp/(tn+fp): 0.159
    
    Positive predicted value tp/(tp+fp): 0.825
    False discovery rate fp/(tp+fp): 0.175
    
    Negative predicted value tn/(tn+fn): 0.841
    False omission rate fn/(tn+fn): 0.159
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -24.000
    Demographic parity tp+fp: 234.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -0.016
    
    Binary proportion

In [4]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "XXXXXXXX_XXXXXX_EfficientNet_Fairface"

with strategy.scope():
    # Load base model trained on FairFace data set without reweighting and without augmentation layers
    model = utils.loadModel(dir_name + "/EfficientNet_FairFace.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

# Crossvalidate model using entire Museum data set
history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (300,300), None, 5, 20)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensor

/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 49 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 49 all-reduces with algorithm = nccl, num_packs = 1


2022-12-16 13:33:00.595506: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-16 13:33:02.303924: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-16 13:33:05.335578: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


31/31 [==============================] - 108s 1s/step - loss: 0.4497 - accuracy: 0.7871 - true_positives_40: 1434.0000 - true_negatives_26: 1649.0000 - false_positives_32: 413.0000 - false_negatives_34: 421.0000 - true_positive_rate: 0.7730 - true_negative_rate: 0.7997 - false_positive_rate: 0.2003 - false_negative_rate: 0.2270 - positive_predicted_value: 0.7764 - false_discovery_rate: 0.2236 - negative_predicted_value: 0.7966 - false_omission_rate: 0.2034 - binary_demographic_parity_diff: -223.0000 - demographic_parity: 1847.0000 - binary_equalized_odds_diff: 0.0267 - binary_proportional_parity_diff: -0.0569 - proportional_parity: 0.4715 - binary_predictive_rate_parity_diff: -0.0202 - predictive_rate_parity: 0.7764 - binary_accuracy_parity_diff: 0.5742 - accuracy_parity: 0.7871 - false_negative_rate_parity_diff: -0.0267 - false_positive_rate_parity_diff: 0.0267 - binary_negative_predictive_rate_parity_diff: 0.0202 - negative_predictive_rate_parity: 0.7764 - binary_specificity_parity_d

In [5]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_EfficientNet"
os.mkdir(dir_name)

# Evaluate the results of the crossvalidation
ev.evaluateCrossValidation(dir_name, history)

Make folder...

    Cross Validation results:
    -------------
    
    Accuracy: 0.927+/-0.011
    Loss: 0.177+/-0.022
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.922+/-0.016
    True negative rate tn/(tn+fp): 0.932+/-0.008
    
    False negative rate fn/(tp+fn): 0.078+/-0.016
    False positive rate fp/(tn+fp): 0.068+/-0.008
    
    Positive predicted value tp/(tp+fp): 0.925+/-0.010
    False discovery rate fp/(tp+fp): 0.075+/-0.010
    
    Negative predicted value tn/(tn+fn): 0.929+/-0.013
    False omission rate fn/(tn+fn): 0.071+/-0.013
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.051+/-0.012
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.010+/-0.011
    
    Binary specificity parity diff tn/(tn+fp)-tp/(tp+fn): -0.010+/-0.011
    
    
    Cross Validation validation results:
    -------------
    
    Accuracy: 0.845+/-0.006
    Loss: 0.403+/-0.024
    
    Tes

## Transfer learning with augmentation and without reweighting
Transfer learning with EfficientNet, with data augmentation and without reweighting. Trained and tested with Museum data set.

In [9]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose the directory where model EfficientNet_FairFace_aug.h5 exists
dir_name = "XXXXXXXX_XXXXXX_EfficientNet_Fairface_aug"

with strategy.scope():
    # Load base model trained on FairFace data set with augmentation layers and without reweighting
    model = utils.loadModel(dir_name + "/EfficientNet_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )
    
# Freeze layers
utils.freezeCertainLayers(model, 338)  
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

# Train model without class weights
utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, None, False)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_transfer_aug"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name_transfer)
utils.saveModel(model, dir_name_transfer + "/EfficientNet_transfer_aug.h5")

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Epoch 1/20
INFO:tensorflow:batch_all_reduce: 46 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 46 all-reduces with algorithm = nccl, num_packs = 1
31/31 [==============================] - 115s 1s/step - loss: 0.6392 - accuracy: 0.7308 - true_positives_284: 1500.0000 - true_negatives_186: 1362.0000 - false_positives_228: 674.0000 - false_negatives_242: 380.0000 - true_positive_rate: 0.7979 - true_negative_rate: 0.6690 - false_positive_rate: 0.3310 - false_negative_rate: 0.2021 - positive_predicted_value: 0.6900 - false_discovery_rate: 0.3100 - negative_predicted_value: 0.7819 - false_omission_rate: 0.2181 - binary_demographic_parity_diff: 432.0000 - demographic_parity: 2174.0000 - binary_equalized_odds_diff: -0.1289 - binary_proportional_parity_diff: 0.110

In [10]:
# Test the model
test_predict, test_labels, dir_name_transfer = ev.testModel(model, ds_test, test_batches, dir_name_transfer)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name_transfer)

Testing Model
-------------
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.831
    
    True positives: 184.000
    False positives: 33.000
    
    True negatives: 225.000
    False negatives: 50.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.786
    True negative rate tn/(tn+fp): 0.872
    
    False negative rate fn/(tp+fn): 0.214
    False positive rate fp/(tn+fp): 0.128
    
    Positive predicted value tp/(tp+fp): 0.848
    False discovery rate fp/(tp+fp): 0.152
    
    Negative predicted value tn/(tn+fn): 0.818
    False omission rate fn/(tn+fn): 0.182
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -58.000
    Demographic parity tp+fp: 217.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -0.086
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.11

In [6]:
strategy = tf.distribute.MirroredStrategy()
# Choose the directory where model EfficientNet_FairFace_aug.h5 exists
dir_name = "XXXXXXXX_XXXXXX_EfficientNet_Fairface_aug"

with strategy.scope():
    # Load base model trained on FairFace data set with augmentation layers and without reweighting
    model = utils.loadModel(dir_name + "/EfficientNet_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

# Crossvalidate model using entire Museum data set
history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (300,300), None, 5, 20)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Load data...
Start crossvalidation...
Fold 0...
Epoch 1/20
31/31 [==============================] - 113s 1s/step - loss: 0.4103 - accuracy: 0.8162 - true_positives_101: 1481.0000 - true_negatives_66: 1716.0000 - false_positives_81: 360.0000 - false_negatives_86: 360.0000 - true_positive_rate: 0.8045 - true_negative_rate: 0.8266 - false_positive_rate: 0.1734 - false_negative_rate: 0.1955 - positive_predicted_value: 0.8045 - false_discovery_rate: 0.1955 - negative_predicted_value: 0.8266 - false_omission_rate: 0.1734 - binary_demographic_parity_diff: -235.0000 - demographic_parity: 1841.0000 - binary_equalized_odds_diff: 0.0221 - binary_proportional_parity_diff: -0.0600 - proportional_parity: 0.4700 - binary_predictive_rate_parity_diff: -0.0221 - predictive_rate_parity: 0.8045 - binary_accuracy_parity_diff: 0.6324 - accuracy_parity: 0.8162 -

In [7]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_EfficientNet_aug"
os.mkdir(dir_name)

# Evaluate the results of the crossvalidation
ev.evaluateCrossValidation(dir_name, history)

Make folder...

    Cross Validation results:
    -------------
    
    Accuracy: 0.893+/-0.019
    Loss: 0.246+/-0.039
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.883+/-0.026
    True negative rate tn/(tn+fp): 0.902+/-0.014
    
    False negative rate fn/(tp+fn): 0.117+/-0.026
    False positive rate fp/(tn+fp): 0.098+/-0.014
    
    Positive predicted value tp/(tp+fp): 0.891+/-0.017
    False discovery rate fp/(tp+fp): 0.109+/-0.017
    
    Negative predicted value tn/(tn+fn): 0.895+/-0.022
    False omission rate fn/(tn+fn): 0.105+/-0.022
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.057+/-0.016
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.020+/-0.013
    
    Binary specificity parity diff tn/(tn+fp)-tp/(tp+fn): -0.020+/-0.013
    
    
    Cross Validation validation results:
    -------------
    
    Accuracy: 0.837+/-0.004
    Loss: 0.386+/-0.020
    
    Tes

## Transfer learning without augmentation and with reweighting
Transfer learning with EfficientNet, with reweighting and without augmentation. Trained and tested with Museum data set.

In [10]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose the directory where model EfficientNet_FairFace_re.h5 exists
dir_name = "XXXXXXXX_XXXXXX_EfficientNet_Fairface_re"

with strategy.scope():
    # Load base model trained on FairFace data set with reweighting and without augmentation layers
    model = utils.loadModel(dir_name + "/EfficientNet_FairFace_re.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Freeze layers   
utils.freezeCertainLayers(model, 334)    
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

# Train model with class weights
utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_transfer_re"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name_transfer)
utils.saveModel(model, dir_name_transfer + "/EfficientNet_transfer_re.h5")

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 49 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 49 all-reduces with algorithm = nccl, num_packs = 1
31/31 [==============================] - 123s 1s/step - loss: 0.6559 - accuracy: 0.6946 - true_positives_122: 1502.0000 - true_negatives_80: 1218.0000 - false_positives_98: 834.0000 - false_negatives_104: 362.0000 - true_positive_rate: 0.8058 - true_negative_rate: 0.5936 - false_positive_rate: 0.4064 - false_negative_rate: 0.1942 - positive_predicted_value: 0.6430 - false_discovery_rate: 0.3570 - negative_predicted_value: 0.7709 - false_omission_rate: 0.2291 - binary_demographic_parity_diff: 756.0000 - demographic_parity: 2336.0000 - binary_equalized_odds_diff: -0.2122 - binary_proportional_parity_diff: 0.1931 - proportional_parity: 0.5965 - binary_predictive_rate_parity_diff: -0.1279 - predictive_rate_parity: 0.6430 - binary_accuracy_parity_diff: 0.3892 - accuracy_parity: 0.6946 - false_negative_rate_pari

In [11]:
# Test the model
test_predict, test_labels, dir_name_transfer = ev.testModel(model, ds_test, test_batches, dir_name_transfer)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name_transfer)

Testing Model
-------------
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.813
    
    True positives: 181.000
    False positives: 39.000
    
    True negatives: 219.000
    False negatives: 53.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.774
    True negative rate tn/(tn+fp): 0.849
    
    False negative rate fn/(tp+fn): 0.226
    False positive rate fp/(tn+fp): 0.151
    
    Positive predicted value tp/(tp+fp): 0.823
    False discovery rate fp/(tp+fp): 0.177
    
    Negative predicted value tn/(tn+fn): 0.805
    False omission rate fn/(tn+fn): 0.195
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -52.000
    Demographic parity tp+fp: 220.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -0.075
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.10

In [4]:
strategy = tf.distribute.MirroredStrategy()
# Choose the directory where model EfficientNet_FairFace_re.h5 exists
dir_name = "XXXXXXXX_XXXXXX_EfficientNet_Fairface_re"

with strategy.scope():
    # Load base model trained on FairFace data set with reweighting and without augmentation layers
    model = utils.loadModel(dir_name + "/EfficientNet_FairFace_re.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

# Crossvalidate model using entire Museum data set
history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (300,300), class_weight, 5, 20)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensor

/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 49 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 49 all-reduces with algorithm = nccl, num_packs = 1


2022-12-16 13:50:41.862281: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-16 13:50:43.247654: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-16 13:50:50.412870: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


31/31 [==============================] - 124s 1s/step - loss: 0.4386 - accuracy: 0.7917 - true_positives_40: 1500.0000 - true_negatives_26: 1601.0000 - false_positives_32: 454.0000 - false_negatives_34: 362.0000 - true_positive_rate: 0.8056 - true_negative_rate: 0.7791 - false_positive_rate: 0.2209 - false_negative_rate: 0.1944 - positive_predicted_value: 0.7677 - false_discovery_rate: 0.2323 - negative_predicted_value: 0.8156 - false_omission_rate: 0.1844 - binary_demographic_parity_diff: -9.0000 - demographic_parity: 1954.0000 - binary_equalized_odds_diff: -0.0265 - binary_proportional_parity_diff: -0.0023 - proportional_parity: 0.4989 - binary_predictive_rate_parity_diff: -0.0479 - predictive_rate_parity: 0.7677 - binary_accuracy_parity_diff: 0.5834 - accuracy_parity: 0.7917 - false_negative_rate_parity_diff: 0.0265 - false_positive_rate_parity_diff: -0.0265 - binary_negative_predictive_rate_parity_diff: 0.0479 - negative_predictive_rate_parity: 0.7677 - binary_specificity_parity_di

In [5]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_EfficientNet_re"
os.mkdir(dir_name)

# Evaluate the results of the crossvalidation
ev.evaluateCrossValidation(dir_name, history)

Make folder...

    Cross Validation results:
    -------------
    
    Accuracy: 0.913+/-0.016
    Loss: 0.206+/-0.036
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.913+/-0.012
    True negative rate tn/(tn+fp): 0.913+/-0.021
    
    False negative rate fn/(tp+fn): 0.087+/-0.012
    False positive rate fp/(tn+fp): 0.087+/-0.021
    
    Positive predicted value tp/(tp+fp): 0.905+/-0.022
    False discovery rate fp/(tp+fp): 0.095+/-0.022
    
    Negative predicted value tn/(tn+fn): 0.921+/-0.011
    False omission rate fn/(tn+fn): 0.079+/-0.011
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.039+/-0.011
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): -0.001+/-0.010
    
    Binary specificity parity diff tn/(tn+fp)-tp/(tp+fn): 0.001+/-0.010
    
    
    Cross Validation validation results:
    -------------
    
    Accuracy: 0.842+/-0.015
    Loss: 0.423+/-0.048
    
    Tes

## Transfer learning with augmentation and reweighting
Transfer learning with EfficientNet, with reweighting and with augmentation. Trained and tested with Museum data set.

In [14]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose the directory where model EfficientNet_FairFace_aug.h5 exists
dir_name = "XXXXXXXX_XXXXXX_EfficientNet_Fairface_aug"

with strategy.scope():
    # Load base model trained on FairFace data set with augmentation layers and without reweighting
    model = utils.loadModel(dir_name + "/EfficinetNet_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Freeze layers   
utils.freezeCertainLayers(model, 334)  
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

# Train model with class weights
utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_transfer_re_aug"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name_transfer)
utils.saveModel(model, dir_name_transfer + "/EfficientNet_transfer_re_aug.h5")

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 49 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 49 all-reduces with algorithm = nccl, num_packs = 1


2022-12-14 14:01:59.134878: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-14 14:02:01.222059: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-14 14:02:04.519696: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


31/31 [==============================] - 116s 1s/step - loss: 0.6186 - accuracy: 0.7375 - true_positives_180: 1502.0000 - true_negatives_117: 1386.0000 - false_positives_144: 666.0000 - false_negatives_153: 362.0000 - true_positive_rate: 0.8058 - true_negative_rate: 0.6754 - false_positive_rate: 0.3246 - false_negative_rate: 0.1942 - positive_predicted_value: 0.6928 - false_discovery_rate: 0.3072 - negative_predicted_value: 0.7929 - false_omission_rate: 0.2071 - binary_demographic_parity_diff: 420.0000 - demographic_parity: 2168.0000 - binary_equalized_odds_diff: -0.1304 - binary_proportional_parity_diff: 0.1073 - proportional_parity: 0.5536 - binary_predictive_rate_parity_diff: -0.1001 - predictive_rate_parity: 0.6928 - binary_accuracy_parity_diff: 0.4750 - accuracy_parity: 0.7375 - false_negative_rate_parity_diff: 0.1304 - false_positive_rate_parity_diff: -0.1304 - binary_negative_predictive_rate_parity_diff: 0.1001 - negative_predictive_rate_parity: 0.6928 - binary_specificity_parit

In [15]:
# Test the model
test_predict, test_labels, dir_name_transfer = ev.testModel(model, ds_test, test_batches, dir_name_transfer)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name_transfer)

Testing Model
-------------
Instructions for updating:
use `experimental_local_results` instead.
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.825
    
    True positives: 194.000
    False positives: 46.000
    
    True negatives: 212.000
    False negatives: 40.000
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.829
    True negative rate tn/(tn+fp): 0.822
    
    False negative rate fn/(tp+fn): 0.171
    False positive rate fp/(tn+fp): 0.178
    
    Positive predicted value tp/(tp+fp): 0.808
    False discovery rate fp/(tp+fp): 0.192
    
    Negative predicted value tn/(tn+fn): 0.841
    False omission rate fn/(tn+fn): 0.159
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -12.000
    Demographic parity tp+fp: 240.000
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.007
    
    Binary proportiona

In [5]:
strategy = tf.distribute.MirroredStrategy()
# Choose the directory where model Xception_FairFace_aug.h5 exists
dir_name = "XXXXXXXX_XXXXXX_Xception_Fairface_aug"

with strategy.scope():
    # Load base model trained on FairFace data set with augmentation layers and without reweighting
    model = utils.loadModel(dir_name + "/EfficientNet_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

# Crossvalidate model using entire Museum data set
history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (300,300), class_weight, 5, 20)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensor

/apps/Arch/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
INFO:tensorflow:batch_all_reduce: 49 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 49 all-reduces with algorithm = nccl, num_packs = 1


2022-12-16 14:10:20.093048: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-16 14:10:21.774727: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2022-12-16 14:10:24.735375: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


31/31 [==============================] - 108s 1s/step - loss: 0.4199 - accuracy: 0.8070 - true_positives_40: 1546.0000 - true_negatives_26: 1615.0000 - false_positives_32: 416.0000 - false_negatives_34: 340.0000 - true_positive_rate: 0.8197 - true_negative_rate: 0.7952 - false_positive_rate: 0.2048 - false_negative_rate: 0.1803 - positive_predicted_value: 0.7880 - false_discovery_rate: 0.2120 - negative_predicted_value: 0.8261 - false_omission_rate: 0.1739 - binary_demographic_parity_diff: 7.0000 - demographic_parity: 1962.0000 - binary_equalized_odds_diff: -0.0245 - binary_proportional_parity_diff: 0.0018 - proportional_parity: 0.5009 - binary_predictive_rate_parity_diff: -0.0381 - predictive_rate_parity: 0.7880 - binary_accuracy_parity_diff: 0.6140 - accuracy_parity: 0.8070 - false_negative_rate_parity_diff: 0.0245 - false_positive_rate_parity_diff: -0.0245 - binary_negative_predictive_rate_parity_diff: 0.0381 - negative_predictive_rate_parity: 0.7880 - binary_specificity_parity_diff

In [6]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_EfficientNet_re_aug"
os.mkdir(dir_name)

# Evaluate the results of the crossvalidation
ev.evaluateCrossValidation(dir_name, history)

Make folder...

    Cross Validation results:
    -------------
    
    Accuracy: 0.890+/-0.005
    Loss: 0.253+/-0.010
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.890+/-0.006
    True negative rate tn/(tn+fp): 0.890+/-0.006
    
    False negative rate fn/(tp+fn): 0.110+/-0.006
    False positive rate fp/(tn+fp): 0.110+/-0.006
    
    Positive predicted value tp/(tp+fp): 0.881+/-0.005
    False discovery rate fp/(tp+fp): 0.119+/-0.005
    
    Negative predicted value tn/(tn+fn): 0.899+/-0.006
    False omission rate fn/(tn+fn): 0.101+/-0.006
    
    Binary proportional parity diff ((tp+fp)/(tp+fp+tn+fn))-((tn+fn)/(tp+fp+tn+fn)): -0.038+/-0.009
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp)): 0.000+/-0.006
    
    Binary specificity parity diff tn/(tn+fp)-tp/(tp+fn): -0.000+/-0.006
    
    
    Cross Validation validation results:
    -------------
    
    Accuracy: 0.839+/-0.014
    Loss: 0.368+/-0.019
    
    Tes